In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone
from typing import List

import ee
import geemap
import geopandas as gpd
import pandas as pd
import pytz

from fwi_predict.constants import FORECAST_TIMES
from fwi_predict.geo.ee import get_gfs

In [2]:
ee.Initialize()

In [3]:
def get_sample_gfs_forecast(sample: ee.Feature,
							              forecast_times: List,
							              gfs: ee.ImageCollection = None) -> ee.FeatureCollection:
	"""Add docstring."""
	if gfs is None:
		gfs = get_gfs()


	# Get times for which we want forecasts.
	sample_idx = sample.get('sample_idx') # Get sample index
	sample_dt = ee.Date(sample.get('sample_dt'))
	day_prior = sample_dt \
		.advance(5.5, 'hour') \
    .advance(-1, 'day') \
    .update(hour=0, minute=0, second=0)
  
	forecast_time_list = ee.List(forecast_times).map(
    lambda hours: day_prior.advance(hours, 'hour').advance(-6, 'hour').millis() # Again adjusting for timezone so forecasts don't overlap with sample time
  )

  # Pre-filter GFS to reduce computation
	forecast_subset = gfs.filterDate( 
    ee.Date(forecast_time_list.sort().getNumber(0)).advance(-1, 'day'), # Earliest forecast initialization time we are interested in 
    day_prior # Want forecasts initialized one day before sample was taken (5:30am IST)
  )

  # Get latest forecast for each forecast (that is at least one day older than sample time)
	def get_latest_forecast_for_time(forecast_time: ee.Number) -> ee.Image:
		"""Get most recent forecast for a given forecast time."""
    # Get frecast for specific time of interest
		subset = forecast_subset \
			.filter(ee.Filter.lt('creation_time', forecast_time)) \
			.filter(ee.Filter.eq('forecast_time', forecast_time))
      
    # Then get most recent forecast
		latest_init_time = subset.aggregate_array('creation_time').sort().get(-1)
	
		return subset.filter(ee.Filter.eq('creation_time', latest_init_time)).first()
  
  
  # Extract forecast values
	forecasts_for_times = ee.ImageCollection(
    forecast_time_list.map(get_latest_forecast_for_time)
  )

	return forecasts_for_times
	
  # Assign metadata to forecast values and cumulative values
	forecast_values = forecasts_for_times \
    .map(lambda img: img.sample(sample.geometry())) \
    .flatten() \
    .map(lambda f: f # Set metadata
      .set('forecast_creation_dt', f.id().slice(0, 10)) # Same as below
      .set('forecast_hour', f.id().slice(11, 14)) # Would be good to make this less hacky
      .set('sample_idx', sample_idx)
    )
	
	# Map each element of forecast_time_list to each feature of forecast_values
	forecast_values_list = forecast_values.toList(forecast_values.size())
	forecast_values = ee.FeatureCollection(
		forecast_values.map(lambda f: f.set('forecast_time', 
			ee.List(forecast_times).get(forecast_values_list.indexOf(f))))
	)

  # Get forecast at time of sample
	sample_dt_rounded = sample_dt \
    .millis() \
    .divide(1000 * 60 * 60) \
    .round() \
    .multiply(1000 * 60 * 60) # Round sample time to nearest hour
	sample_time_forecast = ee.Image(get_latest_forecast_for_time(sample_dt_rounded))

	return sample_time_forecast

In [4]:
ds = gpd.read_file("../data/clean/measurements_with_metadata.geojson")
temp = ds.iloc[3380:3383]
ee_frame = geemap.gdf_to_ee(temp, date='sample_dt', date_format="yyyy-MM-dd'T'HH:mm:ssZ")

In [5]:
out = ee.Date(ee_frame.first().get('sample_dt')).getInfo()
datetime.fromtimestamp(out['value']/1000, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

'2022-04-23 01:13:00'

In [6]:
result = ee_frame.map(lambda f: get_sample_gfs_forecast(f, FORECAST_TIMES))

In [7]:
sample = ee_frame.first()

In [8]:
gfs = get_gfs()

In [9]:
sample_idx = sample.get('sample_idx') # Get sample index
sample_dt = ee.Date(sample.get('sample_dt'))
day_prior = sample_dt \
	.advance(5.5, 'hour') \
	.advance(-1, 'day') \
	.update(hour=0, minute=0, second=0)

forecast_time_list = ee.List(FORECAST_TIMES).map(
	lambda hours: day_prior.advance(hours, 'hour').advance(-6, 'hour').millis() # Again adjusting for timezone so forecasts don't overlap with sample time
)

forecast_subset = gfs.filterDate( 
	ee.Date(forecast_time_list.sort().getNumber(0)).advance(-2, 'day'), # Earliest forecast initialization time we are interested in 
	day_prior # Want forecasts initialized one day before sample was taken (5:30am IST)
)

In [10]:
# Get latest forecast for each forecast (that is at least one day older than sample time)
def get_latest_forecast_for_time(forecast_time: ee.Number) -> ee.Image:
	"""Get most recent forecast for a given forecast time."""
	# Get frecast for specific time of interest
	subset = forecast_subset \
		.filter(ee.Filter.lt('creation_time', forecast_time)) \
		.filter(ee.Filter.eq('forecast_time', forecast_time))
		
	# Then get most recent forecast
	latest_init_time = subset.aggregate_array('creation_time').sort().get(-1)

	return subset.filter(ee.Filter.eq('creation_time', latest_init_time)).first()


In [11]:
times = forecast_time_list.sort().getInfo()
[datetime.fromtimestamp(d/1000, tz=pytz.timezone('Asia/Kolkata')) for d in times]

[datetime.datetime(2022, 4, 20, 14, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 21, 14, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 22, 7, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 22, 14, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 22, 20, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 23, 8, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>),
 datetime.datetime(2022, 4, 23, 14, 30, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)]

### Check date consistency in geo dataset

In [13]:
gfs_data = pd.read_csv("../data/gcs/train/gfs/measurements_with_metadata.csv")
gfs_data = gfs_data.drop(columns=['system:index', '.geo'])

In [14]:
gfs_data

,downward_shortwave_radiation_flux,forecast_creation_dt,forecast_hour,forecast_time,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,sample_idx,specific_humidity_2m_above_ground,temperature_2m_above_ground,total_cloud_cover_entire_atmosphere,total_precipitation_surface,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground
0,95.000000,2.023101e+09,8.0,8,53.642918,79.599998,0,0.018987,28.235132,65.900002,0.5000,1.075740,6.372488
1,870.699951,2.023101e+09,15.0,15,52.671131,76.599998,0,0.019130,28.977930,22.900000,0.0625,-5.206277,3.657776
2,0.960000,2.023101e+09,21.0,21,53.902569,81.400002,0,0.020040,28.810785,6.300000,0.4375,-2.472117,4.348633
3,186.679993,2.023101e+09,33.0,33,49.854088,77.000000,0,0.018997,28.832361,5.000000,0.0625,0.235383,3.516252
4,851.599976,2.023101e+09,39.0,39,46.887844,70.700005,0,0.017774,29.084528,100.000000,0.0000,-4.562368,1.681265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59623,790.400024,2.024030e+09,41.0,sample,35.507526,47.900002,4968,0.013357,30.899103,100.000000,0.0000,-2.688853,6.169089
59624,3696.874931,NaN,NaN,three_day_cum,114.613156,140.637503,4968,0.043290,135.616333,148.531251,0.0000,-3.768496,15.553073
59625,7351.749847,NaN,NaN,seven_day_cum,229.554296,287.168756,4968,0.087843,269.831906,190.481252,0.0000,-15.111245,27.047040
59626,5639.788242,NaN,NaN,same_day_sum,548.166852,1068.243773,4968,0.231249,475.324069,1025.868762,0.0000,-9.406836,34.990070


In [110]:
gfs_data = gfs_data[gfs_data['forecast_hour'].notna()] # Stick to numeric forecast times
gfs_data['creation_time'] = pd.to_datetime(gfs_data['forecast_creation_dt'].astype(int).astype(str), format='%Y%m%d%H', utc=True).dt.tz_convert(pytz.timezone('Asia/Kolkata'))
gfs_data[['creation_time', 'forecast_creation_dt']].head()

,creation_time,forecast_creation_dt
0,2023-10-08 23:30:00+05:30,2.023101e+09
1,2023-10-08 23:30:00+05:30,2.023101e+09
2,2023-10-08 23:30:00+05:30,2.023101e+09
3,2023-10-08 23:30:00+05:30,2.023101e+09
4,2023-10-08 23:30:00+05:30,2.023101e+09


In [ ]:
gfs_data['sample_dt'] = gfs_data['sample_idx'].map(ds.set_index('sample_idx')['sample_dt'])
gfs_data['forecast_time_check'] = gfs_data['creation_time'] + pd.to_timedelta(gfs_data['forecast_hour'], 'hour')
gfs_data[['sample_idx', 'creation_time', 'sample_dt', 'forecast_time_check', 'forecast_hour', 'forecast_time']].head(40)

Times all appear correct to me. Now check subsequent cleaning.

In [113]:
observations_per_measurement = gfs_data.groupby('sample_idx').size()
assert (observations_per_measurement.eq(observations_per_measurement.iloc[0]).all()), (
	"Number of observations per measurement varies."
)

# Reorder columns and rows
front_cols = ['sample_idx', 'forecast_time', 'forecast_creation_dt', 'forecast_hour'] 
gfs_data = gfs_data[front_cols + [col for col in gfs_data.columns if col not in front_cols]]
gfs_data = gfs_data.sort_values(['sample_idx', 'forecast_time'])

# Pivot wide to one observation per measurement
value_cols = gfs_data.columns[~gfs_data.columns.isin(front_cols)].tolist()
gfs_data_wide = gfs_data.pivot(index='sample_idx', columns='forecast_time', values=value_cols)

This also looks correct to me